In [45]:
import numpy as np
import copy
import open3d as o3d
from sklearn.decomposition import PCA
import time

In [2]:
def ransac(pcd, nb_epochs=1, kn = 10) :
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    iter = 0
    points = np.array(pcd.points)
    while (iter < nb_epochs) :
        print(len(points))
        curr_pnt = points[np.random.randint(len(points))]
        print(curr_pnt)
        # TODO Générer un plan à partir de curr_pnt
        [k, idx, _] = pcd_tree.search_knn_vector_3d(curr_pnt, kn)
        print(idx)
        pca = PCA(3)
        pca.fit(points[list(idx)])
        mean = np.mean(pca.components_, axis=1)
        print(mean)
        es = pca.singular_values_
        print(es)
        plane = [es[0],es[1],es[2],es[0]*mean[0]+es[1]*mean[1]+es[2]*mean[2]]
        print(plane)
        o3d.visualization.draw_geometries([pcd],
                                    zoom=0.455,
                                    front=[-0.4999, -0.1659, -0.8499],
                                    lookat=[2.1813, 2.0619, 2.0999],
                                    up=[0.1204, -0.9852, 0.1215])
        # TODO Inlier/outlier 

        iter+=1
    return plane

In [89]:
def ransac(pcd, thresh=0.1,kn=2,epoch=1000) :
    """
    points = np.array(N,3)
    """
    t1 = time.time()
    pts = np.array(pcd.points)
    best_n_inliers = 0
    i =0
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    best_plane = [0,0,0,0]
    best_inlier_mask = np.zeros(len(pts))
    while (i < epoch) :
        # TODO Générer un plan à partir de curr_pnt
        # TODO Inlier/outlier 
        t2 = time.time()
        curr_pnt = pts[np.random.randint(len(pts))]
        [k, idx, _] = pcd_tree.search_knn_vector_3d(curr_pnt, kn)
        pca = PCA(3)
        pca.fit(pts[list(idx)])
        mean = pca.mean_
        normal = pca.components_[2,:]
        d = np.dot(normal,mean)
        plane = [normal[0],normal[1],normal[2],d]
        t3 = time.time()
        dist_pt     = abs(np.dot(normal[:3],pts.T) + d / np.linalg.norm(normal))
        inlier_mask = np.less_equal(dist_pt,thresh)
        t4 = time.time()
        n_inliers = np.sum(inlier_mask)
        if n_inliers> best_n_inliers:
            best_plane = plane
            best_n_inliers = n_inliers
            best_inlier_mask = inlier_mask
        i += 1
        t5 = time.time()
    return best_plane,list(np.where(best_inlier_mask)[0])

In [120]:
def extract_planes(pcd) :
    pcd_plane = pcd.voxel_down_sample(voxel_size=0.1)
    #pcd_plane = copy.deepcopy(pcd)
    planes = []
    inlierss = []
    while(len(planes)<7 and len(pcd_plane.points) > 100) :
        # plane_model, inliers = ransac(pcd_plane,kn = 5, epoch=10000)
        plane_model, inliers = pcd_plane.segment_plane(distance_threshold=0.1,
                                         ransac_n=3,
                                         num_iterations=10000)
        [a, b, c, d] = plane_model
        print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
        inlier_cloud = pcd_plane.select_by_index(inliers)
        inlier_cloud.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
        inlier_cloud.paint_uniform_color([1.0, 0, 0])
        
        outlier_cloud = pcd_plane.select_by_index(inliers, invert=True)
        outlier_cloud.paint_uniform_color([0, 1.0, 0])
        
        # o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
        #                                 zoom=0.8,
        #                                 front=[-0.4999, -0.1659, -0.8499],
        #                                 lookat=[2.1813, 2.0619, 2.0999],
        #                                 up=[0.1204, -0.9852, 0.1215])

        pcd_plane = outlier_cloud
        planes.append(plane_model)
        inlierss.append(inlier_cloud)

        # TODO remove points according to the plane
    
    o3d.visualization.draw_geometries(inlierss+[outlier_cloud],
                                        zoom=0.8,
                                        front=[-0.4999, -0.1659, -0.8499],
                                        lookat=[2.1813, 2.0619, 2.0999],
                                        up=[0.1204, -0.9852, 0.1215])
    return planes

In [121]:
target_pcd = o3d.io.read_point_cloud("hough-plane-python-master/room_scan1.pcd")
extract_planes(target_pcd)

Plane equation: 0.01x + -0.00y + 1.00z + 0.73 = 0
Plane equation: 0.01x + -0.00y + 1.00z + -1.55 = 0
Plane equation: 0.01x + -0.00y + 1.00z + -1.34 = 0
Plane equation: 0.01x + 0.02y + 1.00z + -0.11 = 0
Plane equation: 0.01x + 0.01y + 1.00z + 0.25 = 0
Plane equation: 0.01x + 0.02y + 1.00z + -0.40 = 0
Plane equation: 0.02x + 0.02y + 1.00z + -0.60 = 0


KeyboardInterrupt: 